In [ ]:
#ViT based on ResNet50

In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback

# Define the path to the dataset
dataset_path = r"E:\ML EXTENSION\Archive_2"

# Get file paths and labels from folder structure
file_paths = []
labels = []

for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    if os.path.isdir(label_path):
        for img_file in os.listdir(label_path):
            file_paths.append(os.path.join(label_path, img_file))
            labels.append(label)

# Convert labels to numerical format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(file_paths, labels_encoded, test_size=0.2, random_state=42)

# Function to preprocess images
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator with augmentation
def augmented_data_generator(file_paths, labels, batch_size=32):
    while True:
        for i in range(0, len(file_paths), batch_size):
            batch_paths = file_paths[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]
            batch_images = np.vstack([preprocess_image(path) for path in batch_paths])
            augmented_images = next(datagen.flow(batch_images, batch_size=batch_size, shuffle=False))
            yield augmented_images, np.array(batch_labels)

# Create a Vision Transformer model with fine-tuning
def create_vision_transformer():
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers[-10:]:
        layer.trainable = True  # Unfreeze the last 10 layers

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)  # Dropout 0.7
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)  # Dropout 0.7
    outputs = layers.Dense(len(label_encoder.classes_), activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

# Custom callback for early stopping based on training and validation accuracy difference
class CustomEarlyStopping(Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_acc = logs.get("accuracy")
        val_acc = logs.get("val_accuracy")
        if train_acc and val_acc and (train_acc - val_acc > 0.05):
            print(f"\nStopping early: Training accuracy {train_acc:.4f} exceeds validation accuracy {val_acc:.4f} by more than 5%.")
            self.model.stop_training = True

vit_model = create_vision_transformer()
vit_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

batch_size = 16
epochs = 50

train_generator = augmented_data_generator(X_train, y_train, batch_size=batch_size)
test_generator = augmented_data_generator(X_test, y_test, batch_size=batch_size)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
custom_early_stopping = CustomEarlyStopping()

# Train the model
vit_model.fit(train_generator, steps_per_epoch=len(X_train)//batch_size,
              epochs=epochs, validation_data=test_generator,
              validation_steps=len(X_test)//batch_size, callbacks=[reduce_lr, early_stopping, custom_early_stopping])

# Evaluate the model
test_loss, test_acc = vit_model.evaluate(test_generator, steps=len(X_test)//batch_size)
print(f'Test accuracy: {test_acc}')

# Predict and evaluate results
y_pred = vit_model.predict(test_generator, steps=len(X_test)//batch_size)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test[:len(y_pred_classes)], y_pred_classes))
print(confusion_matrix(y_test[:len(y_pred_classes)], y_pred_classes))


Epoch 1/50
130/650 ━━━━━━━━━━━━━━━━━━━━ 1:17:31 9s/step - accuracy: 0.1005 - loss: 2.9189

C:\Users\vinit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (96714256 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


149/650 ━━━━━━━━━━━━━━━━━━━━ 1:12:42 9s/step - accuracy: 0.1027 - loss: 2.8902

C:\Users\vinit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


527/650 ━━━━━━━━━━━━━━━━━━━━ 11:06 5s/step - accuracy: 0.1550 - loss: 2.6092

C:\Users\vinit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


650/650 ━━━━━━━━━━━━━━━━━━━━ 3648s 6s/step - accuracy: 0.1680 - loss: 2.5606 - val_accuracy: 0.3935 - val_loss: 1.9554 - learning_rate: 1.0000e-04
Epoch 2/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 2548s 4s/step - accuracy: 0.3528 - loss: 2.0045 - val_accuracy: 0.4884 - val_loss: 1.6225 - learning_rate: 1.0000e-04
Epoch 3/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 2420s 4s/step - accuracy: 0.4195 - loss: 1.8035 - val_accuracy: 0.5120 - val_loss: 1.5683 - learning_rate: 1.0000e-04
Epoch 4/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 2368s 4s/step - accuracy: 0.4828 - loss: 1.6586 - val_accuracy: 0.5550 - val_loss: 1.4046 - learning_rate: 1.0000e-04
Epoch 5/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 2367s 4s/step - accuracy: 0.5222 - loss: 1.5043 - val_accuracy: 0.5553 - val_loss: 1.4023 - learning_rate: 1.0000e-04
Epoch 6/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 2330s 4s/step - accuracy: 0.5550 - loss: 1.4238 - val_accuracy: 0.5770 - val_loss: 1.3665 - learning_rate: 1.0000e-04
Epoch 7/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 